__autor__ = Melany Calderón-Osorno

__versión__ = 0.2

__fecha__ = 2025-07-07

__credits__ = Franck Lejzerowicz

#**Kraken2-Bracken Postprocessing**

This tutorial is based on the study by Calderón-Osorno, M., Rojas-Villalta, D., Lejzerowicz, F. et al. (2025), *The influence of depth on the global deep-sea plasmidome*, Scientific Reports, 15, 2959. https://doi.org/10.1038/s41598-025-86098-5.

In this study, we analyzed 81 deep-sea metagenomes to explore the influence of depth on plasmidome composition. This notebook offers a step-by-step guide to post-processing the results obtained from the Kraken2-Bracken pipeline. As shown in Figures 1 and 2, the raw sequencing data used were of consistently high quality.

<table>
  <tr>
    <td style="text-align: center;">
      <img src="https://raw.githubusercontent.com/mecalderon/Tutorial_Summer_Retreat/master/notebooks/1-Kraken2-Bracken/R1.png" width="600"><br>
      <strong>Figure 1:</strong> Quality distribution of Read 1 sequences.
    </td>
    <td style="text-align: center;">
      <img src="https://raw.githubusercontent.com/mecalderon/Tutorial_Summer_Retreat/master/notebooks/1-Kraken2-Bracken/R2.png" width="600"><br>
      <strong>Figure 2:</strong> Quality distribution of Read 2 sequences.
    </td>
  </tr>
</table>


#**Setup notebook environment**

First, we will clone the repository containing the data generated by the Kraken2-Bracken process.

In [ ]:
!git clone https://github.com/mecalderon/Tutorial_Summer_Retreat.git

Cloning into 'Tutorial_Summer_Retreat'...
remote: Enumerating objects: 2382, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 2382 (delta 14), reused 47 (delta 8), pack-reused 2327 (from 1)
Receiving objects: 100% (2382/2382), 26.02 MiB | 6.75 MiB/s, done.
Resolving deltas: 100% (910/910), done.
Updating files: 100% (2384/2384), done.


The following code installs the ete3 library and downloads the NCBI taxonomy database, which is used to retrieve taxonomic information for a given taxID.

In [ ]:
!pip install ete3
from ete3 import NCBITaxa
ncbi = NCBITaxa()
import warnings
warnings.filterwarnings('ignore')

The following code imports the library used for data post-processing.

In [ ]:
import os
import csv
import glob
import pandas as pd

**Functions**

**Taxonomy retrieval**

We define functions to retrieve the taxonomy associated with a given taxID. The *get_desired_ranks* function takes a TaxID and a list of taxonomic ranks, then returns a dictionary mapping each requested rank to its corresponding TaxID from the lineage of the original TaxID. If a rank is missing, it returns a blank placeholder for that rank. This function is designed to be used within the *get_taxonomy* function.

In [ ]:
def get_desired_ranks(taxid, desired_ranks):
    lineage = ncbi.get_lineage(taxid)
    names = ncbi.get_taxid_translator(lineage)
    lineage2ranks = ncbi.get_rank(names)
    ranks2lineage = dict((rank,taxid) for (taxid, rank) in lineage2ranks.items())
    return{'{}_id'.format(rank): ranks2lineage.get(rank, ' ') for rank in desired_ranks}

The *get_taxonomy* function takes a list of NCBI taxonomic IDs (taxids) and returns their corresponding taxonomy information across standard hierarchical ranks: superkingdom, phylum, class, order, family, genus, and species. For each input taxID, it first retrieves the lineage taxIDs for the specified ranks using the helper function *get_desired_ranks*. Then, it translates each of those taxIDs into their scientific names using ete3's get_taxid_translator function. If a particular rank is not available in the lineage, a blank placeholder is added. The final output is a list of lists, where each sublist contains the original taxID followed by the names of the taxonomic ranks in order.

In [ ]:
def get_taxonomy(taxids):
    desired_ranks = ['superkingdom', 'phylum', 'class', 'order', 'family', 'genus', 'species']
    results = list()
    for taxid in taxids:
        results.append(list())
        results[-1].append(str(taxid))
        ranks = get_desired_ranks(taxid, desired_ranks)
        for key, rank in ranks.items():
            if rank != ' ':
                results[-1].append(list(ncbi.get_taxid_translator([rank]).values())[0])
            else:
                results[-1].append(rank)
    return results

The *make_taxonomy_table* function takes two pandas DataFrames as input: bracken_pd, which contains the results from a Bracken analysis indexed by taxonomic IDs, and taxa_pd, which includes the corresponding taxonomy information for each taxID. The function first resets the index of bracken_pd to turn its index (usually including the taxID and other grouping columns) into a regular column-based DataFrame. It then merges this with the taxonomy information (taxa_pd) based on the taxid column. Next, it constructs a formatted taxonomy string in the style of k__Kingdom; p__Phylum; ... by combining taxonomic levels such as kingdom, phylum, class, and so on, for each row. This formatted taxonomy is stored in a new column called Taxon. Finally, the function returns a simplified table with only two columns: Feature ID (originally featureid) and the corresponding Taxon string. This output is useful for linking Bracken output to standardized taxonomy formats for downstream analysis or visualization.

In [ ]:
def make_taxonomy_table(bracken_pd, taxa_pd):
    bracken_taxa_pd = pd.DataFrame(
        bracken_pd.index.tolist(),
        columns = bracken_pd.index.names)
    bracken_taxa_pd = bracken_taxa_pd.merge(taxa_pd, on='taxid')
    levels = ['kingdom', 'phylum', 'class', 'order', 'family', 'genus', 'species']
    bracken_taxa_pd['Taxon'] = [
        '; '.join(['%s__%s' % (level[0], row[level]) for level in levels])
        for _, row in bracken_taxa_pd.iterrows()
    ]
    bracken_taxa_pd = bracken_taxa_pd[['featureid', 'Taxon']]
    bracken_taxa_pd = bracken_taxa_pd.rename(columns={'featureid': 'Feature ID'})
    return bracken_taxa_pd

**Write feature and taxonomy tables**

The *create_dir* function creates the output directory if it does not already exist. The *write_feature_table* function saves a processed version of a Bracken abundance table to a TSV file. The *write_taxonomy_table* function generates a taxonomy-annotated table by combining Bracken abundance data with taxonomic information.

In [ ]:
def create_dir(dir_path):
    if not os.path.isdir(dir_path):
        os.makedirs(dir_path)


def write_feature_table(output_folder, bracken_pd, db):
    db_dir = '%s/data' % output_folder
    create_dir(db_dir)
    bracken_out_pd = bracken_pd.copy()
    bracken_out_pd.index = bracken_pd.index.droplevel(1)
    bracken_out_pd.fillna(0, inplace=True)
    bracken_out_fpo = '%s/bracken_%s.tsv' % (db_dir, db)
    bracken_out_pd.to_csv(bracken_out_fpo, sep='\t')
    print('Written:', bracken_out_fpo)


def write_taxonomy_table(output_folder, bracken_pd, taxa_pd, db):
    db_dir = '%s/taxonomy/bracken_%s' % (output_folder, db)
    create_dir(db_dir)
    bracken_taxa_pd = make_taxonomy_table(bracken_pd, taxa_pd)
    bracken_taxa_fpo = '%s/bracken_%s.tsv' % (db_dir, db)
    bracken_taxa_pd.to_csv(bracken_taxa_fpo, index=False, sep='\t')
    print('Written:', bracken_taxa_fpo)

**Input/Output paths**


**Inputs**

The following code navigates to the Tutorial_Summer_Retreat/data directory, where the Kraken2-Bracken output is stored.

In [ ]:
cd Tutorial_Summer_Retreat/data

/content/Tutorial_Summer_Retreat/data/Tutorial_Summer_Retreat/data


We created a variable named **bracken_dir** to store the path to the Bracken data directory.

In [ ]:
bracken_dir = 'Bracken'

**Outputs**

We created a variable named **Bracken_processing** to store the path to the output directory.

In [ ]:
output_dir = 'Bracken_processing'

**Collect the bracken outputs files**

The following code collects the output files generated by the Kraken2-Bracken process.

In [ ]:
fps = glob.glob('%s/*/*/results.tsv' % bracken_dir)
fps[:3] + fps[-3:]

['Bracken/SRR3967690/gtdb_genomes/results.tsv',
 'Bracken/SRR3967690/k2_standard/results.tsv',
 'Bracken/SRR3967690/gtdb_plasmids/results.tsv',
 'Bracken/ERR599131/gtdb_genomes/results.tsv',
 'Bracken/ERR599131/k2_standard/results.tsv',
 'Bracken/ERR599131/gtdb_plasmids/results.tsv']

**Check how many databases were used**

Show the count of paths (i.e. samples) per database.

In [ ]:
fps_pd = pd.DataFrame(fps, columns=['path'])
fps_pd['database'] = fps_pd['path'].apply(lambda x: x.split('/')[-2])
display(fps_pd.groupby('database').count())

,path
database,
gtdb_genomes,81
gtdb_plasmids,81
k2_standard,81


 Show the count of databases per sample.

In [ ]:
fps_pd['sample_name'] = fps_pd['path'].apply(lambda x: x.split('/')[-3])
display(fps_pd.iloc[:,1:].groupby('sample_name').count())

,database
sample_name,
ERR598944,3
ERR598947,3
ERR598958,3
ERR598960,3
ERR598964,3
...,...
SRR3967690,3
SRR3967700,3
SRR3968061,3


The code reads Bracken output files, organizes them by database, and builds a dictionary of data frames per database. For each file:

* It reads only selected columns (name, taxonomy_id, new_est_reads) and sets a multi-index using name and taxonomy_id.

* It updates a global name2taxon dictionary with the feature name and taxID.

* It renames the column to the sample name and stores the DataFrame in the brackens dictionary under the corresponding database.

In [ ]:
brackens = {}
name2taxon = {}
for db, fp_pd in fps_pd.groupby('database'):
    for _, row in fp_pd.iterrows():
        path = row['path']
        bracken_pd = pd.read_table(
            path,
            usecols=['name', 'taxonomy_id', 'new_est_reads'],
            index_col=['name', 'taxonomy_id'])
        bracken_pd.index.names = ['featureid', 'taxid']
        name2taxon.update(dict(bracken_pd.index.tolist()))
        bracken_pd.columns = [row['sample_name']]
        brackens.setdefault(db, []).append(bracken_pd)

 Show the number of unique name.

 These are collected in the dict name2taxon, which looks like this:

 ```
 {'Aurantimicrobium photophilum': 1987356,
 'Aurantimicrobium minutum': 708131,
 'Aurantimicrobium sp. MWH-Uga1': 2079575, ...

 ```

In [ ]:
print('Number of names across samples and databases:', len(name2taxon))

Number of names across samples and databases: 6558


**Merge the per sample tables per database**

For each of the databases (keys of the brackens dict), a list of tables can be concatenated based on the shared/not shared taxonomy_id (values).

In [ ]:
bracken_pds = {}
for db, bracken_pd in brackens.items():
    bracken_pds[db] = pd.concat(bracken_pd, axis=1)

print(bracken_pds.keys())

dict_keys(['gtdb_genomes', 'gtdb_plasmids', 'k2_standard'])


Show a bit of the tables.

In [ ]:
for db, bracken_pd in bracken_pds.items():
    print()
    print('Database:', db)
    display(bracken_pd.iloc[:10,:5])
    display(bracken_pd.shape)


Database: gtdb_genomes


,,SRR3967690,SRR3963804,SRR3966130,SRR3965592,ERR599125
featureid,taxid,,,,,
Alteromonas macleodii,28108,294526.0,8030.0,175653.0,65477.0,663.0
Alteromonas mediterranea,314275,46839.0,4892.0,75618.0,12739.0,403.0
Alteromonas australica,589873,19207.0,1650.0,71510.0,7940.0,8426.0
Alteromonas abrolhosensis,1892904,12710.0,258.0,7418.0,2577.0,43.0
Alteromonas sp. IB21,2779369,2923.0,122.0,2226.0,621.0,196.0
Alteromonas sp. RKMC-009,2267264,434.0,524.0,153.0,110.0,NaN
Alteromonas sp. KUL42,2480797,194.0,NaN,128.0,70.0,NaN
Alteromonas sp. KUL106,2480799,231.0,NaN,170.0,60.0,NaN
Alteromonas sp. KC3,2795688,65.0,NaN,73.0,29.0,NaN


(5451, 81)


Database: gtdb_plasmids


,,SRR3967690,SRR3963804,SRR3966130,SRR3965592,ERR599125
featureid,taxid,,,,,
Salinimonas iocasae,2572577,2923.0,199.0,2235.0,1065.0,185.0
Alteromonas sp. RKMC-009,2267264,475.0,NaN,344.0,79.0,NaN
Paraglaciecola mesophila,197222,16.0,NaN,NaN,NaN,NaN
Pseudoalteromonas arctica,394751,132.0,NaN,261.0,429.0,174.0
Pseudoalteromonas shioyasakiensis,1190813,64.0,NaN,323.0,440.0,13.0
Pseudoalteromonas sp. NC201,1514074,36.0,NaN,91.0,120.0,25.0
Pseudoalteromonas carrageenovora,227,16.0,NaN,16.0,40.0,NaN
Shewanella aestuarii,1028752,130.0,NaN,98.0,132.0,NaN
Halomonas sulfidaeris,115553,841.0,47.0,798.0,109.0,449.0


(310, 81)


Database: k2_standard


,,SRR3967690,SRR3963804,SRR3966130,SRR3965592,ERR599125
featureid,taxid,,,,,
Alteromonas macleodii,28108,452419.0,18471.0,311084.0,112171.0,3185.0
Alteromonas mediterranea,314275,29844.0,4926.0,53647.0,9445.0,738.0
Alteromonas sp. RKMC-009,2267264,376.0,704.0,132.0,115.0,NaN
Alteromonas sp. RW2A1,1917158,622.0,141.0,871.0,504.0,NaN
Alteromonas sp. MB-3u-76,2058133,444.0,76.0,519.0,383.0,NaN
Alteromonas sp. B31-7,2785913,1573.0,340.0,5906.0,943.0,5025.0
Alteromonas sp. BL110,1714845,23.0,NaN,64.0,NaN,NaN
Alteromonas sp. 009811495,3002962,17.0,NaN,72.0,24.0,NaN
Alteromonas australica,589873,5252.0,273.0,20896.0,2108.0,3343.0


(2077, 81)

 **Make the taxonomy out of the taxonomy_id**

 Collect the taxonomy paths of each taxid.

In [ ]:
taxa = get_taxonomy(set(name2taxon.values()))
print('Number of taxa extracted from the list of taxonomy_id:', len(taxa))

Number of taxa extracted from the list of taxonomy_id: 6551


Turn taxonomy list of lists into a Pandas dataframe.


In [ ]:
taxa_pd = pd.DataFrame(
    taxa,
    columns=[
        'taxid',
        'kingdom',
        'phylum',
        'class',
        'order',
        'family',
        'genus',
        'species'
    ]
)
taxa_pd['taxid'] = taxa_pd['taxid'].astype(int)
print(taxa_pd)

        taxid kingdom             phylum                      class  \
0           7             Pseudomonadota        Alphaproteobacteria   
1      131080             Pseudomonadota         Betaproteobacteria   
2          11             Actinomycetota              Actinomycetes   
3     2588692          Methanobacteriota  Candidatus Methanoliparia   
4     1179670             Actinomycetota              Actinomycetes   
...       ...     ...                ...                        ...   
6546  1736674               Bacteroidota             Flavobacteriia   
6547  2949090             Pseudomonadota         Betaproteobacteria   
6548   196587             Pseudomonadota        Alphaproteobacteria   
6549   655353             Pseudomonadota        Alphaproteobacteria   
6550   655355               Bacteroidota                Bacteroidia   

                            order                          family  \
0                Hyphomicrobiales               Xanthobacteraceae   
1        

The code creates a new DataFrame from the multi-index of bracken_pd, naming the columns after the index levels. It then merges this DataFrame with taxa_pd using the taxid column to add taxonomic information, and finally prints the result.

In [ ]:
bracken_taxa_pd = pd.DataFrame(
    bracken_pd.index.tolist(),
    columns = bracken_pd.index.names)
bracken_taxa_pd = bracken_taxa_pd.merge(taxa_pd, on='taxid')
print(bracken_taxa_pd)

                       featureid    taxid kingdom            phylum  \
0          Alteromonas macleodii    28108            Pseudomonadota   
1       Alteromonas mediterranea   314275            Pseudomonadota   
2       Alteromonas sp. RKMC-009  2267264            Pseudomonadota   
3          Alteromonas sp. RW2A1  1917158            Pseudomonadota   
4       Alteromonas sp. MB-3u-76  2058133            Pseudomonadota   
...                          ...      ...     ...               ...   
2072     Psychromonas ingrahamii   357794            Pseudomonadota   
2073         Microbulbifer celer   435905            Pseudomonadota   
2074           Tolumonas auensis    43948            Pseudomonadota   
2075  Paenarthrobacter aurescens    43663            Actinomycetota   
2076      Sulfurimonas sp. H1576  2672570          Campylobacterota   

                      class              order              family  \
0       Gammaproteobacteria    Alteromonadales    Alteromonadaceae   
1      

**Write outputs**

The code saves the feature table and taxonomy results for each database.

In [ ]:
for db, bracken_pd in bracken_pds.items():
    print()
    print('Database:', db)
    write_feature_table(output_dir, bracken_pd, db)
    write_taxonomy_table(output_dir, bracken_pd, taxa_pd, db)


Database: gtdb_genomes
Written: Bracken_processing/data/bracken_gtdb_genomes.tsv
Written: Bracken_processing/taxonomy/bracken_gtdb_genomes/bracken_gtdb_genomes.tsv

Database: gtdb_plasmids
Written: Bracken_processing/data/bracken_gtdb_plasmids.tsv
Written: Bracken_processing/taxonomy/bracken_gtdb_plasmids/bracken_gtdb_plasmids.tsv

Database: k2_standard
Written: Bracken_processing/data/bracken_k2_standard.tsv
Written: Bracken_processing/taxonomy/bracken_k2_standard/bracken_k2_standard.tsv
